In [1]:
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.optim as optim
#from Code import Neurons
#from Code.ANN import lstmPolicyPredictor, FullyConnected
from Code.envs.MountainCar import LookupPolicy, PassiveEnv
#from Code.SNN import DynNetwork, SequenceWrapper
import time
from collections import OrderedDict

In [2]:

BATCH_SIZE = 64#512
SIM_TIME = 1
USE_JIT = True

device = torch.device('cpu')

env = PassiveEnv(device)

#torch.backends.cudnn.enabled = False


In [9]:
from Code.Networks import Selector, DynNetwork, OuterWrapper, LSTMWrapper, ReLuWrapper, DummyNeuron, make_SequenceWrapper
from Code.NewNeurons import SeqOnlySpike, CooldownNeuron, LIFNeuron, AdaptiveNeuron

base_config = {
    'ALPHA': 0,
    'BETA': 0,
    'OFFSET': 2,
    'ADAPDECAY': 1,
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

heavyside = {
    **base_config,
    'BETA': 1,
}

mem_loop = OrderedDict([
    ('input', 1),
    ('pre_mem', [['input', 'output'], SeqOnlySpike(128, base_config), nn.Linear]),
    ('output', [['pre_mem'], CooldownNeuron(128, heavyside), nn.Linear]),
])

architecture = OrderedDict([
    ('input', 1),
    ('mem_loop', [['input'], make_SequenceWrapper(DynNetwork(mem_loop), USE_JIT), None]),
    ('post_mem', [['input', 'mem_loop'], SeqOnlySpike(128, base_config), nn.Linear]),
    ('output', [['post_mem'], DummyNeuron(1), nn.Linear]),
])

architecturelstm = OrderedDict([ #TODO: update this
    ('input', 3),
    ('obs', [['input'], Selector(0, 2), None]),
    ('probe', [['input'], Selector(2, 1), None]),
    ('lstm', [['obs'], LSTMWrapper(2, 128), None]),
    ('post_mem', [['probe', 'lstm'], ReLuWrapper(128), nn.Linear]),
    ('output', [['post_mem'], DummyNeuron(2), nn.Linear]),
])

forward_ann = OrderedDict([
    ('input', 1),
    ('first_layer', [['input'], ReLuWrapper(128), nn.Linear]),
    ('second_layer', [['first_layer'], ReLuWrapper(128), nn.Linear]),
    ('output', [['second_layer'], DummyNeuron(1), nn.Linear]),
])

lif = {
    **base_config,
    'BETA': 0.9,
}

forward_snn = OrderedDict([
    ('input', 1),
    ('first_layer', [['input'], make_SequenceWrapper(LIFNeuron(128, lif), USE_JIT), nn.Linear]),
    ('second_layer', [['first_layer'], make_SequenceWrapper(LIFNeuron(128, lif), USE_JIT), nn.Linear]),
    ('output', [['second_layer'], DummyNeuron(1), nn.Linear]),
])

forward_adap = OrderedDict([
    ('input', 1),
    ('first_layer', [['input'], make_SequenceWrapper(LIFNeuron(128, lif), USE_JIT), nn.Linear]),
    ('second_layer', [['first_layer'], make_SequenceWrapper(LIFNeuron(64, lif), USE_JIT), nn.Linear]),
    ('adap_layer', [['first_layer'], make_SequenceWrapper(CooldownNeuron(64, heavyside), USE_JIT), nn.Linear]),
    ('output', [['second_layer', 'adap_layer'], DummyNeuron(1), nn.Linear]),
])

In [10]:
#144, 150, 137, 150

#model = OuterWrapper(DynNetwork(architecture), device, USE_JIT)

#model = OuterWrapper(DynNetwork(forward_ann), device, USE_JIT)

#model = OuterWrapper(DynNetwork(forward_snn), device, USE_JIT)

model = OuterWrapper(DynNetwork(forward_adap), device, USE_JIT)


#model = (OuterWrapper(DynNetwork(architecturelstm), device, True))




In [11]:
teacher = LookupPolicy(device)
mse = nn.MSELoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=1e-3)#0.000011e-6

In [12]:
start = time.time()

for i in range(3000):
    model.zero_grad()
    inputs, targets, mask = env.getBatch(BATCH_SIZE)
    if i%100 == 0:
        for p in model.parameters():
            if torch.isnan(p).any():
                raise Exception('Corrupted Model')
    outputs, _ = model(inputs/0.52)
    loss = (mse(outputs.squeeze(dim=2)*0.0234, targets) * mask).sum() / mask.sum()
    loss.backward()
    optimizer.step()
    if i%100 == 0:
        print(loss.item(), (loss/targets.view(-1).var()).item(), i)

print('Total time: ', time.time()-start)



0.0005204004119150341 0.9564611911773682 0
8.280282054329291e-05 0.16329488158226013 100
3.618705886765383e-05 0.06938478350639343 200
2.3129434339352883e-05 0.04346173256635666 300
1.1950374755542725e-05 0.023363661020994186 400
7.960805305629037e-06 0.01547070499509573 500
6.595272225240478e-06 0.012753039598464966 600
6.067212325433502e-06 0.011569429188966751 700
6.0732822930731345e-06 0.011328786611557007 800
4.707947482529562e-06 0.009485835209488869 900
4.337231075624004e-06 0.0085685383528471 1000
4.008578798675444e-06 0.00763543089851737 1100
4.448111212695949e-06 0.008684057742357254 1200
3.789743459492456e-06 0.0072494992054998875 1300
3.867359282594407e-06 0.007080410607159138 1400
3.593664587242529e-06 0.006606528535485268 1500
3.871431545121595e-06 0.007667739875614643 1600
3.3723799788276665e-06 0.006564905866980553 1700
3.017428525708965e-06 0.0058224801905453205 1800
3.2681532502465416e-06 0.00641852430999279 1900
3.003119672939647e-06 0.005801779683679342 2000
3.02697

In [14]:
#model.save('../models/forward_cooldown_passive')

In [13]:
from matplotlib import pyplot as plt
#model = torch.load('../models/snn_passive3')
%matplotlib


inputs, targets, mask = env.getBatch(1)
outputs, _ = model(inputs/0.52)
plt.close()
plt.plot(inputs[:, 0, 0], targets)
plt.plot(inputs[:, 0, 0], outputs.squeeze().detach()*0.0234)

Using matplotlib backend: TkAgg


In [9]:
#TODO: output reset mechanism

config = {
    'ALPHA': 0.7,
    'BETA': 0.9, #0.95
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

secondconfig = {
    'ALPHA': 0.7,
    'BETA': 0.5, #0.95
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

outconfig = {
    'ALPHA': 0,
    'BETA': 0,
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

'''
architecture1 = OrderedDict([
    ('input', [1]),
    ('pre_mem', [64, ['input', 'mem'], Neurons.LIFNeuron, config]),
    ('mem', [32, ['pre_mem'], Neurons.CooldownNeuron, config]), #CooldownNeuron
    ('post_mem', [64, ['input', 'mem'], Neurons.LIFNeuron, config]),
    ('output', [1, ['post_mem'], Neurons.OutputNeuron, outconfig]), #OutputNeuron
])

architecture = OrderedDict([
    ('input', [1]),
    ('pre_mem', [64, ['input', 'mem', 'short_mem'], Neurons.LIFNeuron, config, nn.Linear]),
    ('mem', [32, ['pre_mem'], Neurons.CooldownNeuron, config, nn.Linear]), #CooldownNeuron
    ('short_mem', [32, ['pre_mem'], Neurons.CooldownNeuron, secondconfig, nn.Linear]),
    ('post_mem', [64, ['input', 'mem', 'short_mem'], Neurons.LIFNeuron, config, nn.Linear]),
    ('output', [1, ['post_mem'], Neurons.OutputNeuron, outconfig, nn.Linear]), #OutputNeuron
])


BATCH_SIZE = 64
SIM_TIME = 1


device = torch.device('cpu')

env = PassiveEnv(device)

#model = lstmPolicyPredictor(1,32,64)

#model = FullyConnected([1, 128, 128, 1])

model_raw = DynNetwork(architecture, SIM_TIME)
model = SequenceWrapper(model_raw, BATCH_SIZE, device, False)
'''


"\narchitecture1 = OrderedDict([\n    ('input', [1]),\n    ('pre_mem', [64, ['input', 'mem'], Neurons.LIFNeuron, config]),\n    ('mem', [32, ['pre_mem'], Neurons.CooldownNeuron, config]), #CooldownNeuron\n    ('post_mem', [64, ['input', 'mem'], Neurons.LIFNeuron, config]),\n    ('output', [1, ['post_mem'], Neurons.OutputNeuron, outconfig]), #OutputNeuron\n])\n\narchitecture = OrderedDict([\n    ('input', [1]),\n    ('pre_mem', [64, ['input', 'mem', 'short_mem'], Neurons.LIFNeuron, config, nn.Linear]),\n    ('mem', [32, ['pre_mem'], Neurons.CooldownNeuron, config, nn.Linear]), #CooldownNeuron\n    ('short_mem', [32, ['pre_mem'], Neurons.CooldownNeuron, secondconfig, nn.Linear]),\n    ('post_mem', [64, ['input', 'mem', 'short_mem'], Neurons.LIFNeuron, config, nn.Linear]),\n    ('output', [1, ['post_mem'], Neurons.OutputNeuron, outconfig, nn.Linear]), #OutputNeuron\n])\n\n\nBATCH_SIZE = 64\nSIM_TIME = 1\n\n\ndevice = torch.device('cpu')\n\nenv = PassiveEnv(device)\n\n#model = lstmPolicyPr